<a href="https://colab.research.google.com/github/koojayeong/TextAnalysis/blob/main/Bag_of_Word_IMDB_Review_Deep_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Modules

In [5]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2' :
  reload(sys)
  sys.setdefaultencoding("utf-8")

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 데이터 불러오기

In [6]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/labeledTrainData.tsv", 
                    header=0, # the first line of the file contains column names
                    delimiter="\t", # the fields are separated by tabs
                    quoting=3) # ignore doubled quotes
train = train.drop(['id'], axis=1)
train.head()

,sentiment,review
0,1,"""With all this stuff going down at the moment ..."
1,1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,0,"""The film starts with a manager (Nicholas Bell..."
3,0,"""It must be assumed that those who praised thi..."
4,1,"""Superbly trashy and wondrously unpretentious ..."


In [8]:
import pandas as pd
imdb = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/imdb_master.csv", 
                    encoding="latin-1") 
imdb.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [9]:
imdb = imdb.drop(['Unnamed: 0','type','file'], axis=1)
imdb.columns = ["review", "sentiment"]
imdb.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [10]:
imdb = imdb[imdb.sentiment != 'unsup']
imdb['sentiment'] = imdb['sentiment'].map({'pos':1, 
                                           'neg':0})
imdb.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [11]:
df = pd.concat([train, imdb]).reset_index(drop=True)
df.head()

,sentiment,review
0,1,"""With all this stuff going down at the moment ..."
1,1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,0,"""The film starts with a manager (Nicholas Bell..."
3,0,"""It must be assumed that those who praised thi..."
4,1,"""Superbly trashy and wondrously unpretentious ..."


# 전처리

In [15]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
  text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
  text = text.lower()
  text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
  text = [lemmatizer.lemmatize(token, "v") for token in text]
  text = [word for word in text if not word in stop_words]
  text = " ".join(text)
  return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

In [17]:
df.head()

,sentiment,review,Processed_Reviews
0,1,"""With all this stuff going down at the moment ...",stuff go moment mj ive start listen music watc...
1,1,"""\""The Classic War of the Worlds\"" by Timothy ...",classic war world timothy hines entertain film...
2,0,"""The film starts with a manager (Nicholas Bell...",film start manager nicholas bell give welcome ...
3,0,"""It must be assumed that those who praised thi...",must assume praise film greatest film opera ev...
4,1,"""Superbly trashy and wondrously unpretentious ...",superbly trashy wondrously unpretentious 80 ex...


In [18]:
df.Processed_Reviews.apply(lambda x : len(x.split(" "))).mean()

128.53272

# Keras Model

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train=tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t, y, 
          batch_size=batch_size, 
          epochs=epochs,
          validation_split=0.2)

Epoch 1/3
600/600 [==============================] - 116s 188ms/step - loss: 0.3369 - accuracy: 0.8477 - val_loss: 0.2434 - val_accuracy: 0.9048
Epoch 2/3
600/600 [==============================] - 153s 255ms/step - loss: 0.2154 - accuracy: 0.9177 - val_loss: 0.1669 - val_accuracy: 0.9429
Epoch 3/3
600/600 [==============================] - 163s 271ms/step - loss: 0.1678 - accuracy: 0.9384 - val_loss: 0.1888 - val_accuracy: 0.9316


In [21]:
df_test = pd.read_csv('/content/drive/MyDrive/kaggle/BagsOfPopcorn/testData.tsv',
                      header=0,
                      delimiter='\t',
                      quoting=3)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x : clean_text(x))
df_test["sentiment"]=df_test["id"].map(lambda x : 1 if int(x.strip('"').split("_")[1])>=5 else 0)

y_test = df_test["sentiment"]

list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)

from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

F1-score: 0.9537885265257928
Confusion matrix:


array([[12152,   787],
       [  348, 11713]])